### Import Dependencies

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean

### Calculate Geometric Median for precincts in LA

In [9]:
# https://stackoverflow.com/questions/30299267/geometric-median-of-multidimensional-points
# https://www.pnas.org/content/pnas/97/4/1423.full.pdf

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [47]:
def jsonFileLoader(file):
    #read json to df and transpose
    crime_df = pd.read_json(file).transpose()

    #group by precinct 
    gb = crime_df.groupby('LAPD_Area')    
    split_crime_df = [gb.get_group(x) for x in gb.groups]

    prec_crime_dict = {}

    for i, e in enumerate(split_crime_df):
        prec_crime_np = split_crime_df[i][['LAT','LON']].to_numpy()
        #run geo_med on each  to array to return
        y = geometric_median(prec_crime_np)
        x = str(split_crime_df[i]["LAPD_Area_Name"].values[0])
        prec_crime_dict[x] = y

    return prec_crime_dict


In [19]:
def csvFileLoader(file):
    disp_df = pd.read_csv(file_to_load)["LOCATION"]
    return disp_df

In [48]:
file_to_load = '../json_files/crime_2013.json'

In [50]:
jsonFileLoader(file_to_load)

{'Central': array([34.046268430317454, -118.25044004338203], dtype=object),
 'Rampart': array([34.06107176670108, -118.27332504902962], dtype=object),
 'Southwest': array([34.01838682642745, -118.31555108026427], dtype=object),
 'Hollenbeck': array([34.05261478030283, -118.20472845179633], dtype=object),
 'Harbor': array([33.764045318683266, -118.28659168098801], dtype=object),
 'Hollywood': array([34.09982065823611, -118.32996665828792], dtype=object),
 'Wilshire': array([34.06378822207899, -118.35219919914205], dtype=object),
 'West LA': array([34.05012477342844, -118.43769340517912], dtype=object),
 'Van Nuys': array([34.179259353291485, -118.44623076218687], dtype=object),
 'West Valley': array([34.18730227951422, -118.52254132688999], dtype=object),
 'Northeast': array([34.10753459081063, -118.2436207451411], dtype=object),
 '77th Street': array([33.979073260791495, -118.29696376480926], dtype=object),
 'Newton': array([34.00723219039904, -118.26131148875831], dtype=object),
 'Pac

In [25]:
file_to_load = "../json_files/LA_Certificated_Dispensary_Locations.csv"

In [26]:
csvFileLoader(file_to_load)

0      (33.7258, -118.2855)
1      (33.7744, -118.2629)
2      (33.7805, -118.2523)
3       (33.7816, -118.243)
4      (33.7871, -118.3081)
               ...         
415     (34.307, -118.4695)
416    (34.3075, -118.4614)
417    (34.3099, -118.4274)
418    (34.3106, -118.4755)
419    (34.3106, -118.4793)
Name: LOCATION, Length: 420, dtype: object